# Scipy minimize variational plugin

In this notebook, we briefly introduce the particularization of the `Optimizer` abstract plugin for `scipy.optimize.minimize`.

We assume that you are already familiar with the `Optimize` class. If not, you can access a detailed notebook introducting this class [here](../plugins/junctions_and_optimizers.ipynb).

`ScipyMinimizePlugin` is an `Optimizer` wrapping the `scipy.optimize.minimize` method, thus inheriting from all the underlying minimization algorithms.

The plugin can be instantiated as follows:

In [ ]:
import numpy
from qat.vsolve.optimize import ScipyMinimizePlugin
from scipy.optimize import minimize

## A cobyla minimizer over any number of variables, random initialization, 20 max steps
cobyla = ScipyMinimizePlugin(tol=1e-2, 
                             method="COBYLA", 
                             options={"maxiter": 20})

Lets try to use this plugin to solve a QAOA instance.

In [ ]:
from qat.vsolve.qaoa import MaxCut
import networkx as nx
import matplotlib.pyplot as plt
from qat.qpus import get_default_qpu

qpu = get_default_qpu()

stack = cobyla | qpu

graph = nx.cycle_graph(4)

problem = MaxCut(graph)
job = problem.to_job(2)
circuit = job.circuit
result = stack.submit(job)
print("The maxcut problem:")
print(problem)
print("Final energy:", result.value)
print("The optimization data:")
print(result.meta_data["optimizer_data"])
print("The best set of parameters:")
print(result.meta_data["parameters"])

Notice that the 'optimizer_data' entry of the result's meta_data contains the (stringified) output of scipy's minimize function.

As we can see, 20 iterations are not enough for the optimizer to converge.
Lets try with 200:

In [ ]:
cobyla = ScipyMinimizePlugin(method="COBYLA", 
                             tol=1e-2, 
                             options={"maxiter": 200})
stack = cobyla | qpu
result = stack.submit(job)
print("The maxcut problem:")
print(problem)
print("Final energy:", result.value)
print("The optimization data:")
print(result.meta_data["optimizer_data"])